In [1]:
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [85]:
import requests
import time
import copy
from typing import Dict, Any
import pandas as pd
import numpy as np

from ml_trader.utils import load_config

In [23]:
config = load_config()
tickers = config["tickers"]

In [24]:
filepath = "/Users/kirillgugaev/Dev/finance/ML-shares-trader/datasets/quandl/SHARADAR_TICKERS_6cc728d11002ab9cb99aa8654a6b9f4e.csv"

## Tickers from Quandl (exists on Tinkoff & on Quandl)

In [46]:
tickers_df = pd.read_csv(filepath)
tickers_df[tickers_df['table'] == 'SF1']
tickers_df

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
0,SF1,196290,A,Agilent Technologies Inc,NYSE,N,Domestic Common Stock,00846U101,3826.0,Manufacturing,...,USD,California; U.S.A,2021-06-01,2014-09-26,1999-11-18,2021-07-20,1997-06-30,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,SF1,124392,AA,Alcoa Corp,NYSE,N,Domestic Common Stock,013872106,3334.0,Manufacturing,...,USD,Pennsylvania; U.S.A,2021-05-05,2016-11-01,2016-11-01,2021-07-20,2014-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,SF1,122827,AAAB,Admiralty Bancorp Inc,NASDAQ,Y,Domestic Common Stock,007231103,6022.0,Finance Insurance And Real Estate,...,USD,Florida; U.S.A,2019-07-29,2017-09-09,1998-09-25,2003-01-28,1997-09-30,2002-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
3,SF1,120538,AAAGY,Altana Aktiengesellschaft,NYSE,Y,ADR Common Stock,02143N103,2834.0,Manufacturing,...,EUR,Jordan,2019-05-17,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
4,SF1,155760,AAAP,Advanced Accelerator Applications SA,NASDAQ,Y,ADR Common Stock,00790T100,2834.0,Manufacturing,...,EUR,France,2020-10-08,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2017-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49001,SF3B,635775,NaN,LONG WALK MANAGEMENT LP,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,NaN,NaN,NaN,NaN,NaN,2021-06-30,2021-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
49002,SF3B,635630,NaN,CARDINAL POINT WEALTH MANAGEMENT LLC,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,NaN,NaN,NaN,NaN,NaN,2020-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
49003,SF3B,635623,NaN,CORSICANA CO,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,NaN,NaN,NaN,NaN,NaN,2021-06-30,2021-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
49004,SF3B,635648,NaN,BANK OF NEW HAMPSHIRE,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,NaN,NaN,NaN,NaN,NaN,2021-06-30,2021-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


In [47]:
tmp = pd.DataFrame()
tmp['ticker'] = tickers
tmp['flag'] = True
tmp.head()

,ticker,flag
0,A,True
1,AA,True
2,AAL,True
3,AAN,True
4,AAON,True


In [48]:
tickers_df = pd.merge(tickers_df, tmp, on='ticker', how='left')
tickers_df['flag'] = tickers_df['flag'].fillna(False)
tickers_df = tickers_df[tickers_df['flag']]
del tickers_df['flag']

tickers_df.reset_index(drop=True)

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
0,SF1,196290,A,Agilent Technologies Inc,NYSE,N,Domestic Common Stock,00846U101,3826.0,Manufacturing,...,USD,California; U.S.A,2021-06-01,2014-09-26,1999-11-18,2021-07-20,1997-06-30,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,SF1,124392,AA,Alcoa Corp,NYSE,N,Domestic Common Stock,013872106,3334.0,Manufacturing,...,USD,Pennsylvania; U.S.A,2021-05-05,2016-11-01,2016-11-01,2021-07-20,2014-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,SF1,199960,AAL,American Airlines Group Inc,NASDAQ,N,Domestic Common Stock,02376R102,4512.0,Transportation Communications Electric Gas And...,...,USD,Texas; U.S.A,2021-04-22,2015-04-14,2013-12-09,2021-07-20,2012-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.aa.com
3,SF1,632687,AAN,Aaron's Company Inc,NYSE,N,Domestic Common Stock,00258W108,7359.0,Services,...,USD,Georgia; U.S.A,2021-05-02,2020-12-02,2020-11-25,2021-07-20,2018-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
4,SF1,198259,AAON,Aaon Inc,NASDAQ,N,Domestic Common Stock,000360206,3585.0,Manufacturing,...,USD,Oklahoma; U.S.A,2021-05-06,2014-10-14,1992-12-16,2021-07-20,1996-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.aaon.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,SEP,194904,ZUMZ,Zumiez Inc,NASDAQ,N,Domestic Common Stock,989817101,5600.0,Retail Trade,...,USD,Washington; U.S.A,2021-07-20,2014-10-27,2005-05-06,2021-07-20,2003-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zumiez.com
2990,SEP,119847,ZUO,Zuora Inc,NYSE,N,Domestic Common Stock,98983V106,7372.0,Services,...,USD,California; U.S.A,2021-07-20,2018-04-14,2018-04-12,2021-07-20,2016-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zuora.com
2991,SEP,634860,ZY,Zymergen Inc,NASDAQ,N,Domestic Common Stock,98985X100,8731.0,Services,...,USD,California; U.S.A,2021-07-20,2021-04-22,2021-04-22,2021-07-20,2019-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zymergen.com
2992,SEP,157146,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,N,Domestic Common Stock,98986X109,2834.0,Manufacturing,...,USD,Pennsylvania; U.S.A,2021-07-20,2016-04-15,2015-08-05,2021-07-20,2013-12-31,2021-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zynerba.com


In [52]:
tickers_quandl = tickers_df['ticker'].unique().tolist()
len(tickers_quandl), len(tickers)

(1502, 1549)

In [53]:
with open('tmp.txt', 'w') as f:
    f.write('"' + '", "'.join(tickers_quandl) + '"')

## Tickers (batch)

In [57]:
def _format_quandl_url(path: str) -> str:
    config = load_config()
    return "https://www.quandl.com/api/v3/{}&api_key={}".format(path, config["quandl"]["api_key"])

In [80]:
path = 'datatables/SHARADAR/SF1?ticker={ticker}'
full_url = _format_quandl_url(path.format(ticker=','.join(tickers[:2])))

In [81]:
r = requests.get(full_url)
if r.status_code != 200:
    print(f'Error: {full_url}')

data = r.json()
data

{'datatable': {'data': [],
  'columns': [{'name': 'ticker', 'type': 'String'},
   {'name': 'dimension', 'type': 'String'},
   {'name': 'calendardate', 'type': 'Date'},
   {'name': 'datekey', 'type': 'Date'},
   {'name': 'reportperiod', 'type': 'Date'},
   {'name': 'lastupdated', 'type': 'Date'},
   {'name': 'accoci', 'type': 'Integer'},
   {'name': 'assets', 'type': 'Integer'},
   {'name': 'assetsavg', 'type': 'Integer'},
   {'name': 'assetsc', 'type': 'Integer'},
   {'name': 'assetsnc', 'type': 'Integer'},
   {'name': 'assetturnover', 'type': 'BigDecimal(15,3)'},
   {'name': 'bvps', 'type': 'BigDecimal(15,3)'},
   {'name': 'capex', 'type': 'Integer'},
   {'name': 'cashneq', 'type': 'Integer'},
   {'name': 'cashnequsd', 'type': 'Integer'},
   {'name': 'cor', 'type': 'Integer'},
   {'name': 'consolinc', 'type': 'Integer'},
   {'name': 'currentratio', 'type': 'BigDecimal(15,3)'},
   {'name': 'de', 'type': 'BigDecimal(15,3)'},
   {'name': 'debt', 'type': 'Integer'},
   {'name': 'debtc', '

In [68]:
datatable_data = np.array(data['datatable']['data'])
datatable_data

array([['TSLA', 'MRY', '2018-12-31', ..., '0', '34.455', '-1686000000'],
       ['TSLA', 'MRY', '2017-12-31', ..., '0', '34.016', '-1104150000'],
       ['TSLA', 'MRY', '2016-12-31', ..., '0', '30.91', '432791000'],
       ...,
       ['AAPL', 'MRY', '2013-12-31', ..., '0', '7.767', '29628000000'],
       ['AAPL', 'MRY', '2012-12-31', ..., '0', '6.522', '19111000000'],
       ['AAPL', 'MRY', '2011-12-31', ..., '0', '4.325', '17018000000']],
      dtype='<U32')

In [82]:
ticker_seq = np.array([x[0] for x in data['datatable']['data']])
ticker_seq

array([], dtype=float64)

In [72]:
base_path = '/Users/kirillgugaev/Dev/finance/ML-shares-trader/datasets/quandl/core_fundamental'

In [78]:
exist_tickers = [x.split('.')[0] for x in os.listdir(base_path)]
exist_tickers

['TSLA', 'AAPL']

In [83]:
tickers_to_download = list(set(tickers).difference(set(exist_tickers)))
tickers_to_download

['AJRD',
 'BDX',
 'CVLT',
 'KALU',
 'TPR',
 'AERI',
 'AFX@DE',
 'EQT',
 'PEP',
 'VG',
 'BWXT',
 'STLD',
 'IFX@DE',
 'BLUE',
 'USPH',
 'ABC',
 'CARS',
 'GL',
 'MO',
 'FLS',
 'PLCE',
 'LEGH',
 'MXL',
 'TRUP',
 'ESE',
 'BLNK',
 'ISRG',
 'CRL',
 'JKHY',
 'SHI',
 'ARCT',
 'SFIX',
 'IART',
 'GDOT',
 'NWE',
 'RAVN',
 'ACMR',
 'AFG',
 'TTE',
 'PNFP',
 'FAST',
 'NBIX',
 'U',
 'KWR',
 'EEFT',
 'VICR',
 'CAH',
 'HYLN',
 'RL',
 'REGN',
 'HBAN',
 'JELD',
 'MOH',
 'ALG',
 'APH',
 'LNT',
 'AX',
 'ENS',
 'ODFL',
 'VEEV',
 'ZION',
 'HAL',
 'CE',
 'AEO',
 'EMN',
 'EXC',
 'RHM@DE',
 'AZO',
 'GD',
 'MVIS',
 'RH',
 'GDRX',
 'BRKR',
 'AXON',
 'AIN',
 'HAE',
 'UNF',
 'AVNS',
 'CTSH',
 'FFIV',
 'USFD',
 'WK',
 'SNBR',
 'DY',
 'CWST',
 'CSWI',
 'HQY',
 'KSU',
 'NSC',
 'WEC',
 'COUR',
 'MGY',
 'DELL',
 'FB',
 'GHC',
 'GNRC',
 'EXLS',
 'MLM',
 'MS',
 'EGRX',
 'BLKB',
 'BKNG',
 'PINS',
 'ARQT',
 'NTES',
 'PEGA',
 'CREE',
 'TTMI',
 'VLO',
 'INTU',
 'BIO',
 'GTHX',
 'WTTR',
 'HE',
 'YUM',
 'WSM',
 'DOCU',
 'IGMS',


In [94]:
ticker_data = copy.deepcopy(data)
ticker = tickers[0]
ticker_data['datatable']['data'] = datatable_data[ticker_seq == ticker].tolist() if len(ticker_seq) else []
ticker_data

{'datatable': {'data': [],
  'columns': [{'name': 'ticker', 'type': 'String'},
   {'name': 'dimension', 'type': 'String'},
   {'name': 'calendardate', 'type': 'Date'},
   {'name': 'datekey', 'type': 'Date'},
   {'name': 'reportperiod', 'type': 'Date'},
   {'name': 'lastupdated', 'type': 'Date'},
   {'name': 'accoci', 'type': 'Integer'},
   {'name': 'assets', 'type': 'Integer'},
   {'name': 'assetsavg', 'type': 'Integer'},
   {'name': 'assetsc', 'type': 'Integer'},
   {'name': 'assetsnc', 'type': 'Integer'},
   {'name': 'assetturnover', 'type': 'BigDecimal(15,3)'},
   {'name': 'bvps', 'type': 'BigDecimal(15,3)'},
   {'name': 'capex', 'type': 'Integer'},
   {'name': 'cashneq', 'type': 'Integer'},
   {'name': 'cashnequsd', 'type': 'Integer'},
   {'name': 'cor', 'type': 'Integer'},
   {'name': 'consolinc', 'type': 'Integer'},
   {'name': 'currentratio', 'type': 'BigDecimal(15,3)'},
   {'name': 'de', 'type': 'BigDecimal(15,3)'},
   {'name': 'debt', 'type': 'Integer'},
   {'name': 'debtc', '